In [1]:
import importlib
import sys, os
sys.path.append('C:\\Users\\bartm\\Documents\\These\\phyloreplica\\src')
from PhyloDataset import *

from PhyloTrees import *

from vae import *

import torch
torch.autograd.set_detect_anomaly(True)

datapath1 = "../data/PF00072/PF00072_rp15_has_PF00196.faa"
datapath2 = "../data/PF00072/PF00072_rp15_has_PF00486.faa"
datapath3 = "../data/PF00072/PF00072_rp15_has_PF00512.faa"
datapath4 = "data/PF00072/PF00072_rp15_has_PF00158.faa"
datapath5 = "data/PF00072/PF00072_rp15_has_PF00990.faa"
datapath6 = "data/PF00072/PF00072_rp15_has_PF01339.faa"
datapath7 = "data/PF00072/PF00072_rp15_has_PF04397.faa"
datapath8 = "data/PF00072/PF00072_rp15_has_PF12833.faa"

lossfn = vae_loss

dataset1 = MSA(datapath1)
dataset2 = MSA(datapath2)
dataset3 = MSA(datapath3)
lt = len(dataset1) + len(dataset2) + len(dataset1)
l1 = int(32*len(dataset1)/lt) 
l2 = int(32*len(dataset2)/lt) 
l3 = 32 - l1 -l2

vae1 = VAE(21, 5, dataset1.len_protein * dataset1.q, [512, 256, 128])
optimizer1 = optim.Adam(vae1.parameters(),weight_decay=0.01)
Node1 = PhyloNode(vae1,
          optimizer1, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset1, 
          tuplesize=2, 
          batch_size=l1, 
          gammaManager = gammaManager_Independant(),
          Name = "196"
    )


vae2 = VAE(21, 5, dataset2.len_protein * dataset2.q, [512, 256, 128])
optimizer2 = optim.Adam(vae2.parameters(),weight_decay=0.01)
Node2 = PhyloNode(vae2,
          optimizer2, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset2, 
          tuplesize=2, 
          batch_size=l2, 
          gammaManager = gammaManager_Independant(),
          Name="486"
    )


vae3 = VAE(21, 5, dataset3.len_protein * dataset3.q, [512, 256, 128])
optimizer3 = optim.Adam(vae3.parameters(),weight_decay=0.01)
Node3 = PhyloNode(vae3,
          optimizer3, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset3, 
          tuplesize=2, 
          batch_size=l3, 
          gammaManager = gammaManager_Independant(),
          Name="512"
    )

vaeR =  VAE(21, 5, dataset3.len_protein * dataset3.q, [512, 256, 128])
optimizerR = optim.Adam(vaeR.parameters(),weight_decay=0.01)
NodeR = PhyloNode(vaeR,
          optimizer3, 
          lossfn,
          parent=None, 
          children=[], 
          tuplesize=2, 
          batch_size=32, 
          gammaManager = gammaManager_Independant(),
          Name="Root"
    )


In [18]:
NodeR.addChildren(Node1)
NodeR.addChildren(Node2)
NodeR.addChildren(Node3)

In [19]:
recursive=True
NodeR.getNewTrainBatch(fullBatch=False)
NodeR.trainmode(recursive=recursive)
NodeR.computeLoss(recursive=recursive)
NodeR.computeCouplingLossChildren(recursive=recursive)

tensor(8537.2959, grad_fn=<AddBackward0>)

In [20]:
NodeR.model.encoder_linears[0].weight
NodeR.coupling_loss_Children

tensor(8537.2959, grad_fn=<AddBackward0>)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NodeR.to_(device, recursive=True)

In [ ]:
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer1, factor=0.5, patience=10, verbose=True
)

In [21]:
trainL = int(0.8 * len(dataset1))
testL = int(0.1 * len(dataset1))
valL = len(dataset1) - trainL -testL
batch_size = 64
train_set, test_set,  val_set = torch.utils.data.random_split(dataset1, [trainL, testL, valL])
train_iterator = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_iterator = DataLoader(test_set, batch_size=batch_size, shuffle=True)
val_iterator = DataLoader(val_set, batch_size=batch_size, shuffle=True)


In [60]:
for epoch in range(100):
    print(epoch)
    vae1.train()
    train_loss_list = []
    for batch_idx, batch in enumerate(train_iterator):
        x,w = batch[0], batch[1]
        #targ = targ.unsqueeze(1)
        loss = -vae1.compute_weighted_elbo(x, w)
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
        train_loss_list.append(loss.item())
    
#         l2_reg = torch.tensor(0.)
#         for param in Dcla.parameters():
#             l2_reg += torch.norm(param)
#         loss += l2_lambda * l2_reg

    print(" acc train" ,np.mean(train_loss_list))
#     scheduler.step(np.mean(train_loss_list))
    test_loss_list = []
    with  torch.no_grad():
        for batch_idx, batch in enumerate(test_iterator):
            x,w = batch[0], batch[1]
            loss = -vae1.compute_weighted_elbo(x, w)
            test_loss_list.append(loss.item())
    print(" acc val" ,np.mean(test_loss_list))


0
 acc train 232.64005801213642
 acc val 216.81131757967438
1
 acc train 221.08789975881538
 acc val 216.8724232900696
2
 acc train 220.78262209176722
 acc val 216.28017581379268
3
 acc train 221.00017979834374
 acc val 216.22171973935576
4
 acc train 218.22384155384952
 acc val 210.8948803907571
5
 acc train 213.40594894706072
 acc val 207.86210863053896
6
 acc train 209.83776014020359
 acc val 206.54911125779657
7
 acc train 207.53771635573955
 acc val 204.90933909563287
8
 acc train 206.46169362608848
 acc val 203.47775012884782
9
 acc train 204.993235802025
 acc val 201.76293855947455
10
 acc train 203.13573688453093
 acc val 201.08824371574417
11
 acc train 201.41997401246047
 acc val 199.39542016606822
12
 acc train 200.41088235460992
 acc val 198.73058363782368
13
 acc train 199.7113907324935
 acc val 198.38729480262143
14
 acc train 199.21865676814903
 acc val 198.65687004912672
15
 acc train 198.80171879379918
 acc val 199.12513417776984
16
 acc train 198.69112326924932
 acc v

In [ ]:
a = torch.tensor(1)
b = torch.tensor(2)
torch.min((a,2))

# kmeans Splitter

In [ ]:
dataset1.sequences.shape
len(dataset1)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=5, random_state=0).fit(dataset1.sequences)
kmeans.labels_
    #array([1, 1, 1, 0, 0, 0], dtype=int32)
# kmeans.predict([[0, 0], [12, 3]])
    #array([1, 0], dtype=int32)
kmeans.cluster_centers_
#     array([[10.,  2.],
#            [ 1.,  2.]])

In [ ]:
sum(kmeans.labels_==4)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(dataset1.sequences)

In [ ]:
import matplotlib.pyplot as plt
X_r = pca.transform(dataset1.sequences)
colors = ["navy", "turquoise", "darkorange", "red", "blue"]
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], kmeans.labels_):
    plt.scatter(
        X_r[kmeans.labels_ == i, 0], X_r[kmeans.labels_ == i, 1], color=color, alpha=0.8, lw=lw, label=target_name
    )
plt.legend(loc="best", shadow=False, scatterpoints=1)
plt.title("PCA of IRIS dataset")

plt.figure()

plt.show()

In [ ]:
sub = torch.utils.data.Subset(dataset1, kmeans.labels_==4)

In [ ]:
sub[:][0].shape

In [ ]:
a = torch.tensor(1)

In [ ]:
a.to(a.device)

# Meta Gamma

In [30]:
batch = next(iter(val_iterator))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
def vae_loss(model, batch):
    seq = batch[0]
    weights = batch[1]
    return -1*model.compute_weighted_elbo(seq, weights)

In [32]:

vae1 = VAE(21, 5, dataset1.len_protein * dataset1.q, [512, 256, 128])
optimizer1 = optim.Adam(vae1.parameters(),weight_decay=0.01)
Node1 = PhyloNode(vae1,
          optimizer1, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset1, 
          tuplesize=2, 
          batch_size=l1, 
          gammaManager = gammaManager_Independant(),
          Name = "196"
    )


vae2 = VAE(21, 5, dataset2.len_protein * dataset2.q, [512, 256, 128])
optimizer2 = optim.Adam(vae2.parameters(),weight_decay=0.01)
Node2 = PhyloNode(vae2,
          optimizer2, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset2, 
          tuplesize=2, 
          batch_size=l2, 
          gammaManager = gammaManager_Independant(),
          Name="486"
    )


In [26]:
replicas = self.replicas
Y = self.Y

replica_state_dicts = [replicas[y].state_dict() for y in range(Y)]
replica_mean_state_dict = OrderedDict()

for key in replicas[Y].state_dict().keys():
    replica_mean_state_dict[key] = \
        torch.mean(torch.stack([state_dict[key] for state_dict in replica_state_dicts]), 0).detach()

replicas[Y].load_state_dict(replica_mean_state_dict)


NameError: name 'self' is not defined

In [33]:
import copy
vae1 = vae1.to(device)
vae2 = vae2.to(device)


gamma=torch.tensor(0.1, requires_grad=True)
optimizerGamma = optim.Adam(vaeR.parameters(),weight_decay=0.01)
mylearningModel = copy.deepcopy(vae1)

# for p in mylearningModel.parameters():
#     p.requires_grad=False


In [34]:
 mylearningModel.parameters()

<generator object Module.parameters at 0x000001E0851CD580>

In [91]:
vae1

VAE(
  (encoder_linears): ModuleList(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (encoder_mu): Linear(in_features=128, out_features=5, bias=True)
  (encoder_logsigma): Linear(in_features=128, out_features=5, bias=True)
  (decoder_linears): ModuleList(
    (0): Linear(in_features=5, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=2352, bias=True)
  )
)

In [35]:

mylearningModel = VAE(21, 5, dataset1.len_protein * dataset1.q, [512, 256, 128])
import copy
vae1 = vae1.to(device)
vae2 = vae2.to(device)
vae1.eval()

gamma =torch.tensor(0.1, requires_grad=True)

# mylearningModel = copy.deepcopy(vae1)
optimizerGamma = optim.Adam(mylearningModel.parameters())
print("before",mylearningModel.encoder_linears[1].weight)
for p1,p2,pt in zip(vae1.parameters(), vae2.parameters(), mylearningModel.parameters()):
    pt.data = p1.data.clone().detach() + gamma * p2.data.clone().detach()
print("after",mylearningModel.encoder_linears[1].weight)

loss = vae_loss(mylearningModel,batch)
print("loss", loss)
loss.backward()
print(gamma.grad)


before Parameter containing:
tensor([[ 0.0316,  0.0246, -0.0102,  ...,  0.0235, -0.0160, -0.0120],
        [ 0.0049, -0.0327,  0.0039,  ...,  0.0314,  0.0055, -0.0130],
        [-0.0426, -0.0048,  0.0108,  ...,  0.0152, -0.0345,  0.0260],
        ...,
        [ 0.0038,  0.0261,  0.0352,  ..., -0.0382,  0.0087,  0.0032],
        [ 0.0396,  0.0232,  0.0269,  ..., -0.0109,  0.0322, -0.0409],
        [-0.0045, -0.0162, -0.0167,  ..., -0.0276,  0.0189,  0.0029]],
       requires_grad=True)
after Parameter containing:
tensor([[ 0.0161,  0.0031, -0.0433,  ..., -0.0086, -0.0327,  0.0414],
        [ 0.0336,  0.0307, -0.0148,  ...,  0.0176, -0.0044,  0.0324],
        [-0.0125, -0.0222, -0.0311,  ..., -0.0359,  0.0257, -0.0188],
        ...,
        [-0.0231,  0.0213,  0.0113,  ..., -0.0319, -0.0403, -0.0168],
        [ 0.0052,  0.0372,  0.0307,  ..., -0.0283, -0.0111,  0.0348],
        [ 0.0155, -0.0379,  0.0376,  ...,  0.0086, -0.0198, -0.0292]],
       device='cuda:0', requires_grad=True)
loss

In [41]:
list(mylearningModel.parameters())[1].grad

tensor([ 1.4603e-03,  5.7922e-03, -6.8263e-03, -1.4060e-03, -4.1660e-03,
         9.8772e-04,  8.5723e-03, -1.2746e-03,  3.4750e-03, -4.5513e-03,
         4.0655e-04, -5.7472e-03, -7.8497e-03, -2.8538e-03,  7.4794e-03,
        -2.0981e-04,  2.2235e-03,  1.9217e-03,  1.9003e-03,  4.8133e-03,
        -3.0225e-03, -5.4674e-03, -7.6896e-03,  4.3949e-03,  2.3638e-03,
        -1.2172e-03, -5.9228e-03, -2.1191e-03,  6.2634e-04,  5.8063e-04,
         4.7241e-03, -8.0270e-03,  6.2557e-03, -8.4195e-03, -1.6472e-03,
         6.4508e-03,  1.1457e-03, -3.9582e-03, -6.4723e-03, -7.3779e-03,
         8.7304e-04,  4.8071e-03,  5.3020e-04, -7.1232e-04, -6.1115e-03,
         1.7270e-03,  6.0199e-03,  4.0438e-03,  3.5270e-04,  5.2790e-03,
        -1.8656e-03,  4.0551e-04, -6.8779e-03,  5.0000e-03, -2.9553e-03,
         3.1306e-03,  3.7630e-03, -7.6335e-03, -2.4267e-03,  9.8965e-04,
        -5.3717e-03,  2.4834e-03, -4.6136e-03,  7.2253e-03,  3.3006e-03,
         3.1863e-03,  1.0567e-02,  4.1016e-03,  5.9

In [11]:
params = list(vae2.parameters())
params[1].grad

In [120]:
optimizerGamma.zero_grad()
print(mylearningModel)
print(torch.autograd.grad(loss, mylearningModel.encoder_linears[1].weight, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=True))

VAE(
  (encoder_linears): ModuleList(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (encoder_mu): Linear(in_features=128, out_features=5, bias=True)
  (encoder_logsigma): Linear(in_features=128, out_features=5, bias=True)
  (decoder_linears): ModuleList(
    (0): Linear(in_features=5, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=2352, bias=True)
  )
)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [55]:
def gammaGradient(model1, model2, modelRobot, batch, gamma):
    modelRobot.zero_grad()
    for p1,p2,pt in zip(model1.parameters(), model2.parameters(), mylearningModel.parameters()):
        pt.data = p1.data.clone().detach() + gamma * (p2.data.clone().detach() - p1.data.clone().detach())
    loss = vae_loss(modelRobot,batch)
    loss.backward()
    paramsmodelRobot = list(modelRobot.parameters())
    paramsmodel1 = list(model1.parameters())
    paramsmodel2 = list(model2.parameters())
    grad = 0
    for i in range(len(paramsmodel1)):
        dif = (paramsmodel2[i].data - paramsmodel1[i].data)*paramsmodelRobot[i].grad
        grad += torch.sum(dif)
    return grad

    
    

In [59]:
gammaGradient(vae1, vae2, mylearningModel, batch, 0.0)

tensor(-2.5507, device='cuda:0')

In [53]:
mylearningModel.zero_grad()

In [5]:
grad = OrderedDict()

NameError: name 'OrderedDict' is not defined

In [117]:
vae1.eval()
loss = vae_loss(vae1,batch)
print("loss", loss)

loss tensor(341.7444, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)


In [88]:
mylearningModel.encoder_linears[1].weight.grad


tensor([[-4.5299e-04, -4.1638e-04,  5.0943e-04,  ..., -4.3712e-04,
         -5.2714e-04,  2.2105e-05],
        [ 2.7423e-04,  1.8226e-04, -9.2665e-05,  ...,  3.7737e-04,
         -2.1523e-05,  1.3462e-04],
        [-3.5861e-04, -3.1159e-04,  9.2236e-04,  ..., -5.1846e-04,
          1.7147e-04, -2.4603e-04],
        ...,
        [ 8.9274e-05,  3.3852e-04,  6.9726e-04,  ...,  1.3256e-04,
          1.7706e-04,  6.5539e-05],
        [ 2.3475e-04,  5.4092e-04, -1.4460e-03,  ..., -6.9047e-04,
         -1.0559e-03,  9.4670e-04],
        [-6.8411e-04, -2.9299e-04,  6.4204e-04,  ..., -1.0027e-03,
         -9.2636e-04,  1.6536e-04]], device='cuda:0')

In [53]:
for pt in mylearningModel.parameters():
    pt.data = 0*pt.data



In [56]:
for pt in mylearningModel.parameters():
    print(pt)


Parameter containing:
tensor([[-0., -0., -0.,  ..., -0., 0., 0.],
        [-0., -0., 0.,  ..., -0., -0., -0.],
        [0., -0., 0.,  ..., -0., -0., 0.],
        ...,
        [-0., -0., 0.,  ..., -0., -0., 0.],
        [-0., 0., -0.,  ..., 0., -0., 0.],
        [0., -0., -0.,  ..., 0., -0., 0.]], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([-0., 0., 0., -0., 0., -0., 0., 0., -0., -0., -0., 0., 0., 0., -0., 0., 0., 0., 0., -0., 0., 0., 0., 0.,
        -0., -0., 0., -0., -0., -0., 0., 0., 0., 0., -0., 0., 0., 0., 0., 0., 0., -0., 0., 0., -0., 0., 0., -0.,
        -0., 0., -0., 0., 0., -0., -0., -0., -0., -0., -0., 0., -0., -0., 0., 0., 0., 0., 0., 0., -0., 0., 0., -0.,
        0., -0., -0., -0., 0., 0., 0., 0., -0., -0., -0., 0., 0., -0., 0., 0., 0., 0., 0., 0., -0., -0., -0., 0.,
        -0., 0., 0., 0., -0., 0., -0., -0., -0., -0., 0., 0., -0., -0., -0., -0., 0., 0., -0., -0., 0., 0., -0., 0.,
        0., 0., -0., -0., 0., -0., 0., 0., 0., -0., -0., -0., 0.

In [64]:
loss

tensor(341.8337, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)

In [66]:
print(gamma.grad)

None


In [ ]:
class gammaManager_auto(nn.Module):
    def __init__(self):
        super(gammaManager_Independant, self).__init__()
        self.gammaParents = torch.tensor(0.0)
        self.gammaChildren = torch.tensor(0.0)
        self.timestep = torch.tensor(0.0)
        
    def composeLoss(self, Node):
        return Node.loss + self.gammaParents * Node.coupling_loss_Parents + self.gammaChildren * Node.coupling_loss_Children
    
    def updateGamma(self,Node):
            self.timestep +=1
    def reinitGamma(self, Node):
        self.gammaParents = torch.tensor(0.0).to(self.gammaParents.device)
        self.gammaChildren = torch.tensor(0.0).to(self.gammaChildren.device)
    def to_(self,device):
        self.gammaParents = self.gammaParents.to(device)
        self.gammaChildren = self.gammaChildren.to(device)
        self.timestep = self.timestep.to(device)

# GPT

In [1]:
import importlib
import sys, os
sys.path.append('C:\\Users\\bartm\\Documents\\These\\phyloreplica\\src')
from PhyloDataset import *
from PhyloTrees import *

from vae import *
from GProT import *
import torch
torch.autograd.set_detect_anomaly(True)

datapath1 = "../data/PF00072/PF00072_rp15_has_PF00196.faa"
datapath2 = "../data/PF00072/PF00072_rp15_has_PF00486.faa"
datapath3 = "../data/PF00072/PF00072_rp15_has_PF00512.faa"
datapath4 = "data/PF00072/PF00072_rp15_has_PF00158.faa"
datapath5 = "data/PF00072/PF00072_rp15_has_PF00990.faa"
datapath6 = "data/PF00072/PF00072_rp15_has_PF01339.faa"
datapath7 = "data/PF00072/PF00072_rp15_has_PF04397.faa"
datapath8 = "data/PF00072/PF00072_rp15_has_PF12833.faa"

lossfn = vae_loss

dataset1 = MSA(datapath1)

In [3]:
config = GPTConfig(21, 112)
co = GPT1Config(21, 112)
co.n_layer = 3
co.n_head = 7
co.n_embd = 50*21

In [4]:
dataset1.device

In [5]:
dataset1[1][0].shape

torch.Size([112, 21])

In [6]:
model = GPT(co, dataset1.device)

cpu


In [7]:
train_iterator = DataLoader(dataset1, batch_size=32, shuffle=True)

In [8]:
batch = next(iter(train_iterator))

In [9]:
sequences = batch[0]
sequences.shape

torch.Size([32, 112, 21])

In [12]:
a = model(sequences)

In [15]:
a[0].shape

torch.Size([32, 112, 21])

In [19]:
a[1]

In [36]:
criterion = nn.CrossEntropyLoss()

In [41]:
import torch.nn.functional as F
inp = batch[0].reshape(-1, batch[0].shape[2])
tar = batch[0].max(dim=2)[1].flatten()
F.cross_entropy(inp*10, tar)

tensor(0.0009)

In [ ]:
torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

In [ ]:
torch.nn.functional.

In [23]:
batch[0].max(dim=2)[1]

tensor([[17,  9, 17,  ...,  0, 17, 14],
        [ 7, 17,  7,  ...,  0, 17, 14],
        [17,  0,  7,  ...,  0,  7, 14],
        ...,
        [17,  9, 17,  ...,  0,  7, 14],
        [17,  7, 17,  ..., 15,  7, 20],
        [ 7,  9,  7,  ...,  6,  7, 20]])

In [ ]:
inp = batch[0].reshape(-1, 21)
tar = batch[0].max(dim=2)[1].flatten()

In [33]:
inp[0,:].unsqueeze(0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0.]])

In [29]:
tar[0].unsqueeze(0)

tensor(17)

In [34]:
F.cross_entropy(inp[0,:].unsqueeze(0), tar[0].unsqueeze(0), weight=None, size_average=None, ignore_index=- 100, reduce=None, reduction='mean', label_smoothing=0.0)

tensor(2.1232)

In [38]:
criterion(inp[0,:].unsqueeze(0)*10, tar[0].unsqueeze(0))

tensor(0.0009)